In [15]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from deem import deem # proposed module, deem is in short for 'de'-biasing 'em'beddings

In [16]:
classf_param = {'C': [10**k for k in range(-10, 4, 1)], 
                'metric': 'roc_auc', 'cv': 3}  

In [17]:
def debias(irmas_train, irmas_test, openmic_train, openmic_test, openmic_mask):
    
    X_train, X_test = deem.projection(irmas_train, irmas_test)
    train, test = (X_train, irmas_train[1]), (X_test, irmas_test[1])
    irmas_irmas = deem.irmas_irmas(train, test)

    X_train, X_test = deem.projection(openmic_train, openmic_test)
    train, test = (X_train, openmic_train[1]), (X_test, openmic_test[1])
    openmic_openmic = deem.openmic_openmic(train, test, openmic_mask)
    
    X_train, X_test = deem.projection(irmas_train, openmic_test)
    train, test = (X_train, irmas_train[1]), (X_test, openmic_test[1])
    irmas_openmic = deem.irmas_openmic(train, test, openmic_mask)
    
    X_train, X_test = deem.projection(openmic_train, irmas_test)
    train, test = (X_train, openmic_train[1]), (X_test, irmas_test[1])
    openmic_irmas = deem.openmic_irmas(train, test)
    
    result = pd.concat([irmas_irmas, openmic_openmic, irmas_openmic, openmic_irmas], ignore_index=True)
    result['embedding'] = [deem.embedding + deem.debias_method] * len(result)

    print(deem.debias_method)
    return result

# VGGish

In [18]:
embedding = 'vggish'
deem = deem(embedding, classf_param)

irmas_train, irmas_test = deem.data_loader(dataset='irmas', data_root='')
openmic_train, openmic_test, openmic_mask = deem.data_loader(dataset='openmic', data_root='openmic-2018/')

Loading irmas data:


  0%|          | 0/6705 [00:00<?, ?it/s]

Loading openmic data:


  0%|          | 0/20000 [00:00<?, ?it/s]

In [32]:
deem.debias_method = ''  # '' = orginal, no debiasing
result_all = debias(irmas_train, irmas_test, openmic_train, openmic_test, openmic_mask)  # original

In [33]:
debias_methods = ['', '-lda', '-lda-genre', '-k', '-klda', '-klda-genre']

for method in debias_methods[1:]:
    deem.debias_method = method 
    result_all = result_all.append(debias(irmas_train, irmas_test, openmic_train, openmic_test, openmic_mask))

-lda
-lda-genre
-k
-klda
-klda-genre


# OpenL3

In [7]:
embedding = 'openl3'
deem.embedding = embedding

irmas_train, irmas_test = deem.data_loader(dataset='irmas', data_root='')
openmic_train, openmic_test, openmic_mask = deem.data_loader(dataset='openmic', data_root='openmic-2018/')

Loading irmas data:


  0%|          | 0/6705 [00:00<?, ?it/s]

Loading openmic data:


  0%|          | 0/20000 [00:00<?, ?it/s]

In [8]:
debias_methods = ['', '-lda', '-lda-genre', '-k', '-klda', '-klda-genre']

for method in debias_methods:
    deem.debias_method = method 
    result_all = result_all.append(debias(irmas_train, irmas_test, openmic_train, openmic_test, openmic_mask))


-lda
-lda-genre
-k
-klda
-klda-genre


# YAMNet

In [9]:
embedding = 'yamnet'
deem.embedding = embedding

irmas_train, irmas_test = deem.data_loader(dataset='irmas', data_root='')
openmic_train, openmic_test, openmic_mask = deem.data_loader(dataset='openmic', data_root='openmic-2018/')

Loading irmas data:


  0%|          | 0/6705 [00:00<?, ?it/s]

Loading openmic data:


  0%|          | 0/20000 [00:00<?, ?it/s]

In [10]:
for method in debias_methods:
    deem.debias_method = method
    result_all = result_all.append(debias(irmas_train, irmas_test, openmic_train, openmic_test, openmic_mask))


-lda
-lda-genre
-k
-klda
-klda-genre


In [31]:
result_all.to_csv('results/result_all.csv', index=False)  